In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='domain'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_1215652/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  MtnClimber                50      1   
                     SeekAndFind               24      1   
                     george76                   2      1   
apnews.com           E. Pluribus Unum           3      1   
babylonbee.com       george76                   2      1   
                     E. Pluribus Unum           2      1   
                     Morgana                    3      1   
                     marcusmaximus              1      1   
bbc.com              MtnClimber                 1      1   
                     marcusmaximus              2      1   
                     Red Badger                 2      1   
                     nickcarraway               9      1   
breitbart.com        george76                   2      1   
                     E. Pluribus Unum           1      1   
                     Morgana                   27      1   
                     marcusmaximus              1      1   
                     Red Badger                 2      1   
                     ChicagoConservative27    182      1   
dailymail.co.uk      SeekAndFind                2      1   
                     george76                   2      1   
                     E. Pluribus Unum           1      1   
                     Morgana                   88      1   
                     marcusmaximus              5      1   
                     Red Badger                 7      1   
                     nickcarraway               4      1   
                     Libloather                 4      1   
foxnews.com          george76                   7      1   
                     E. Pluribus Unum           1      1   
                     Morgana                    1      1   
                     marcusmaximus              1      1   
                     Red Badger                 1      1   
                     ChicagoConservative27      2      1   
                     Libloather                16      1   
frontpagemag.com     MtnClimber                 3      1   
                     george76                   6      1   
                     E. Pluribus Unum          11      1   
justthenews.com      E. Pluribus Unum          12      1   
latimes.com          E. Pluribus Unum          25      1   
                     Red Badger                 1      1   
                     ChicagoConservative27      2      1   
msn.com              SeekAndFind                7      1   
                     E. Pluribus Unum           1      1   
                     marcusmaximus              2      1   
                     Red Badger                 1      1   
                     nickcarraway               1      1   
                     ChicagoConservative27      2      1   
newsweek.com         george76                   1      1   
                     E. Pluribus Unum           3      1   
                     marcusmaximus              3      1   
                     Red Badger                 1      1   
                     nickcarraway               4      1   
                     ChicagoConservative27      1      1   
nypost.com           MtnClimber                 2      1   
                     george76                  10      1   
                     E. Pluribus Unum           3      1   
                     Morgana                    1      1   
                     marcusmaximus             14      1   
                     Red Badger                 1      1   
                     nickcarraway              14      1   
                     ChicagoConservative27     31      1   
                     Libloather                82      1   
nytimes.com          E. Pluribus Unum          49      1   
pjmedia.com          SeekAndFind               19      1   
                     george76                   6      1   
                     Re

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_1215652/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


timestamp  \
title                                                                    
Steve Bannon: “I will have Elon Musk run out of... 2025-01-13 09:38:57   
Supreme Court leans in favor of state-enforced ... 2025-01-16 10:52:22   
New Fire in LA, Hollywood Hills!                   2025-01-09 05:48:08   
Hegseth confirmed 51-50.                           2025-01-25 03:54:12   
Trump had ‘horrendous’ call with Danish PM over... 2025-01-24 22:47:52   
Heartbroken Hollywood actor whose house burnt d... 2025-01-08 14:04:37   
Steve Bannon Goes Nuclear on ‘Truly Evil’ and ‘... 2025-01-12 14:34:39   

                                                    replies  
title                                                        
Steve Bannon: “I will have Elon Musk run out of...    283.0  
Supreme Court leans in favor of state-enforced ...    201.0  
New Fire in LA, Hollywood Hills!                      199.0  
Hegseth confirmed 51-50.                              176.0  
Trump had ‘horrendous’ call with Danish PM over...    166.0  
Heartbroken Hollywood actor whose house burnt d...    157.0  
Steve Bannon Goes Nuclear on ‘Truly Evil’ and ‘...    156.0